In [ ]:
import sys
import cv2
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.model_selection import train_test_split

IMAGE_SIZE = 64 # size
IMAGE_CHANNEL = 3 #RGB
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*IMAGE_CHANNEL
NUM_CLASSES = 2

target_dir = 'cute/' # 可愛い顔画像
other_dir = 'other/' # 上記以外

# 顔認識用のカスケード型識別器の読み込み
cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def load_face_img(file, target_size):
    # 画像ファイル読み込み
    img = cv2.imread(file)
    
    # グレースケール変換
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 顔領域の探索
    face = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(60, 60))

    # 検出部分の切り出し。最初に検出した１つだけを切り出す（１画像に顔１つが前提）
    if len(face):
        x, y, w, h = face[0]
        face_img = img[y:y+h, x:x+w]
    else:
        face_img = img
    
    # サイズの変更
    face_img = cv2.resize(face_img, target_size)
    
    return np.asarray(face_img)

x=[]
y=[]
    
# load images for train
# - for target images
for picture in list_pictures(target_dir, ext='jpg'):　#拡張子'jpg'ファイルを指定しています
    img = img_to_array(load_face_img(picture, target_size=(IMAGE_SIZE, IMAGE_SIZE)))
    x.append(img)
    y.append(1) # 正解ラベル

# - for other images
for picture in list_pictures(other_dir, ext='jpg'): #拡張子'jpg'ファイルを指定しています
    img = img_to_array(load_face_img(picture, target_size=(IMAGE_SIZE, IMAGE_SIZE)))
    x.append(img)
    y.append(0) # その他ラベル

# arrayに変換
x = np.asarray(x)
y = np.asarray(y)

# 画素値を0から1の範囲に変換
x = x.astype('float32')
x = x / 255.0

# ラベルをカテゴリに変換
y = keras.utils.np_utils.to_categorical(y, NUM_CLASSES)

#Kerasのバックエンドで動くTensorFlowとTheanoでは入力チャンネルの順番が違うので場合分けして書いています
if K.image_data_format() == 'channels_first':
    x = x.reshape(x.shape[0], IMAGE_CHANNEL, IMAGE_SIZE, IMAGE_SIZE)
    input_shape = (IMAGE_CHANNEL, IMAGE_SIZE, IMAGE_SIZE)
else:
    x = x.reshape(x.shape[0], IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL)
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL)

# テストデータ作成（評価用なので、今回はテストデータ未作成）
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.0, random_state=211)


In [ ]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    #rotation_range=180,
    zoom_range=0.3,
    #vertical_flip=True,
    horizontal_flip=True,
    shear_range=0.39, # pi/8
    channel_shift_range=100,
    samplewise_center=True,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    data_format=K.image_data_format())

datagen.fit(x_train)

In [ ]:
# ハイパーパラメータ
batch_size = 10
epochs = 50
learning_rate = 1e-5

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

#display model summary
model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

In [ ]:
# モデルの保存
model_json = model.to_json()
open('model.json', 'w').write(model_json) # モデル
model.save_weights('model.h5'); # 重み